In [2]:
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [3]:
!pip install tweetnlp pandas openpyxl tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [4]:
from tqdm import tqdm
import pandas as pd
import tweetnlp

In [5]:
df = pd.read_excel('socialmedia_topic_singlelabel.xlsx')

In [6]:
print(df.head)

<bound method NDFrame.head of                         id                                                url  \
0      3601950890291405824           https://www.instagram.com/p/DH8s0AyOKRB/   
1      3601850527853700096           https://www.instagram.com/p/DH8V_i-xeRy/   
2      3601417251762960896           https://www.instagram.com/p/DH6zejLS3lC/   
3      3599250906404357120           https://www.instagram.com/p/DHzG6E1vAJN/   
4      3599051737610549248           https://www.instagram.com/p/DHyZnych3hT/   
...                    ...                                                ...   
10868  1910693138961670912  https://x.com/Ministerio_TIC/status/1910693138...   
10869  1910663182097540096   https://x.com/D8peloz/status/1910663182097539468   
10870  1910651920529714944  https://x.com/michaelbedoya99/status/191065192...   
10871  1910533845964701952  https://x.com/Sandra_ForeroR/status/1910533845...   
10872  1911074041420549888  https://x.com/coguensis/status/191107404142055...  

In [7]:
model = tweetnlp.load_model('emotion')

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/configuration_auto.py:1097: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:897: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [8]:
def classify_emotion(text):
    if not isinstance(text, str) or not text.strip():
        return pd.Series({
            'emotion':    None,
            'p_anger':    0.0,
            'p_joy':      0.0,
            'p_optimism': 0.0,
            'p_sadness':  0.0,
        })
    out = model.emotion(text, return_probability=True)
    probs = out['probability']
    return pd.Series({
        'emotion':    out['label'],
        'p_anger':    probs.get('anger',    0.0),
        'p_joy':      probs.get('joy',      0.0),
        'p_optimism': probs.get('optimism', 0.0),
        'p_sadness':  probs.get('sadness',  0.0),
    })


In [9]:
emotion_results = df['content_en'].apply(classify_emotion)

In [10]:
df_out = pd.concat([df, emotion_results], axis=1)
df_out.to_excel('socialmedia_with_emotion.xlsx', index=False)

In [11]:
print(df_out[['content_en', 'emotion', 'p_joy']].head())

                                          content_en       emotion     p_joy
0  Work hard for something that does not interest...      optimism  0.032887
1  Beautiful things that happen in Bogotá._X000D_...           joy  0.950656
2  Relive the first chapter of the 3rd season of ...           joy  0.503901
3  In this profile we also join the trends we mov...  anticipation  0.347138
4  ✈️ A trip between books and heights!✨ _x000D_\...           joy  0.794312


In [12]:
files.download('socialmedia_with_emotion.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>